In [2]:
import numpy
import pandas
import glob
from keras.models import Sequential
from keras.layers import *
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy import misc
import numpy as np

Using TensorFlow backend.


In [58]:
seed = 7
# testImg = glob.glob("data/validation/People/*.jpg") + glob.glob("data/validation/Not People/*.jpg")
numpy.random.seed(seed)
img_width = 100
img_height = 100
epochs = 50

In [59]:
numImg = glob.glob("headtracker_sequences/*/img***.bmp")
datasetX = []
for num in numImg:
    getImg = misc.imread(num)
    getImg = misc.imresize(getImg, (img_width, img_height))
    getImg = np.divide(getImg, 255)
    datasetX.append(getImg)
#print(datasetX)
datasetY = []
ellipses = pandas.read_fwf("seq_dp/ellipse_states.txt").as_matrix()
for num in range(len(numImg)):
    datasetY.append([int(ellipses[num][0].split()[1]), ellipses[num][1],ellipses[num][2]])

ellipses = pandas.read_fwf("seq_dp/ellipse_states.txt").as_matrix()
for num in range(len(numImg)):
    datasetY.append([int(ellipses[num][0].split()[1]), ellipses[num][1],ellipses[num][2]])

datasetX = np.array(datasetX)
datasetY = np.array(datasetY)

[['0  68' 43 17]
 ['1  69' 46 17]
 ['2  69' 46 17]
 ['3  64' 42 17]
 ['4  64' 42 17]
 ['5  64' 41 18]
 ['6  69' 42 18]
 ['7  77' 42 17]
 ['8  86' 42 18]
 ['9  96' 43 17]
 ['10 104' 44 16]
 ['11 104' 43 15]
 ['12  99' 40 15]
 ['13  92' 39 15]
 ['14  86' 40 15]
 ['15  83' 39 15]
 ['16  81' 39 16]
 ['17  83' 38 16]
 ['18  86' 38 17]
 ['19  89' 40 17]
 ['20  92' 42 18]
 ['21  90' 42 18]
 ['22  87' 40 17]
 ['23  82' 40 17]
 ['24  76' 40 18]
 ['25  71' 38 19]
 ['26  66' 35 19]
 ['27  67' 37 20]
 ['28  67' 40 21]
 ['29  67' 44 22]
 ['30  68' 42 22]
 ['31  67' 40 23]
 ['32  63' 38 24]
 ['33  56' 37 24]
 ['34  47' 44 24]
 ['35  28' 48 24]
 ['36  25' 49 25]
 ['37  28' 53 24]
 ['38  25' 52 24]
 ['39  25' 48 24]
 ['40  25' 44 23]
 ['41  25' 42 22]
 ['42  26' 42 22]
 ['43  31' 41 21]
 ['44  36' 40 20]
 ['45  35' 37 19]
 ['46  33' 36 18]
 ['47  32' 32 18]
 ['48  34' 32 18]
 ['49  38' 34 18]
 ['50  45' 36 19]
 ['51  50' 40 20]
 ['52  57' 41 20]
 ['53  62' 41 19]
 ['54  67' 42 19]
 ['55  71' 42 18]
 [

In [61]:
model = Sequential()

model.add(Convolution2D(64,(3,3),input_shape=(img_width,img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Convolution2D(128,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dense(3))

In [62]:
model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [63]:
model.fit(datasetX, datasetY, epochs=epochs, batch_size=64)

Epoch 1/50
94/94 [==============================] - 4s - loss: 1997.7722 - acc: 0.6702     
Epoch 2/50
94/94 [==============================] - 3s - loss: 22113.3857 - acc: 0.6383     
Epoch 3/50
94/94 [==============================] - 3s - loss: 584.7723 - acc: 0.8723     
Epoch 4/50
94/94 [==============================] - 3s - loss: 338.1605 - acc: 0.8723     
Epoch 5/50
94/94 [==============================] - 3s - loss: 204.4874 - acc: 0.8723     
Epoch 6/50
94/94 [==============================] - 3s - loss: 199.7265 - acc: 0.8723     

In [55]:
test_img = misc.imread("seq_dp/img010.bmp")
test_img = np.array([misc.imresize(test_img, (100, 100))])
print(model.predict(test_img, 
                     batch_size=1))

[[ 12313.99609375   7826.45410156   2817.05883789]]


In [13]:
print(model.evaluate(testX, 
                     testY, 
                     batch_size=1))

1174/1181 [============================>.] - ETA: 0s 

In [14]:
model.save("test.h5")

In [24]:
test_img = misc.imread("/Users/frank/Desktop/Github/AR-Duel/data/Validation/Not People/IMG02367.jpg")
print(model.evaluate(np.array([misc.imresize(test_img, (100, 100))]), 
                     np.array([1]), 
                     batch_size=1))

1/1 [==============================] - 0s
[16.118095397949219, 0.0]
